### **<p align="center">🚧 Desarrollo de funciones para API 🚧</p>**

1) def **PlayTimeGenre**( genero : str ): Debe devolver año con mas horas jugadas para dicho género.

importacion de librerias necesarias:

In [1]:
from fastapi import FastAPI, HTTPException
import pandas as pd

In [5]:
app = FastAPI()

In [7]:
# Leer archivos Parquet
df_gamesf1 = pd.read_parquet(r'DataApi/games_f1.parquet')
df_itemsf1 = pd.read_parquet(r'DataApi/items_new.parquet')

In [37]:
df_reviewsf1 = pd.read_parquet(r'DataApi/reviews_parquet')

In [3]:
df_gamesf1.head(1)

,genres,release_date,id
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",2018-01-04,761140.0


In [4]:
df_itemsf1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 2 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           int32  
 1   playtime_forever  float32
dtypes: float32(1), int32(1)
memory usage: 39.3 MB


In [4]:
def PlayTimeGenre(genero: str):
    # Filtro el DataFrame df_games por el género especificado
    df_filtrado = df_gamesf1[df_gamesf1['genres'].str.contains(genero, case=False, na=False)]

    if not df_filtrado.empty:
        # Uno df_filtrado con df_items en item_id para obtener las horas jugadas
        df_merged = pd.merge(df_filtrado, df_itemsf1, left_on='id', right_on='item_id', how='inner')

        # Encontrar el año con más horas jugadas para el género
        año_max_horas = df_merged.groupby('release_date')['playtime_forever'].sum().idxmax()

        return {"Año de lanzamiento con más horas jugadas para género " + genero: año_max_horas}
    else:
        return {"mensaje": "No se encontraron juegos para el género especificado"}

In [5]:
PlayTimeGenre('Simulation')

{'Año de lanzamiento con más horas jugadas para género Simulation': '2006-11-29'}

In [10]:
# Endpoint para obtener el año con más horas jugadas para un género específico
@app.get("/playtime_genre/{genero}")
async def get_playtime_genre(genero: str):
    try:
        return PlayTimeGenre(genero)
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

2) def **UserForGenre**( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

* realizo transformaciones necesarias para poder desarrollar la funcion:

In [38]:

df_gamesf1['release_date'].head(20)

0     2018-01-04
1     2018-01-04
2     2017-07-24
3     2017-12-07
4           None
5     2018-01-04
6     2018-01-04
7     2018-01-04
8     2018-01-04
9     2018-01-04
10        Soon..
11          None
12    2018-01-04
13    2018-01-04
14    2018-01-03
15    2018-01-03
16    2018-01-03
17    2018-01-03
18    2018-01-03
19          None
Name: release_date, dtype: object

In [39]:
# Convierte la columna 'release_date' a formato datetime
pd.to_datetime(df_gamesf1['release_date'], errors='coerce')

0       2018-01-04
1       2018-01-04
2       2017-07-24
3       2017-12-07
4              NaT
           ...    
32130   2018-01-04
32131   2018-01-04
32132   2018-01-04
32133   2017-09-02
32134          NaT
Name: release_date, Length: 32135, dtype: datetime64[ns]

In [40]:
df_gamesf1['release_date'].isnull().sum()

2067

* leo el archivo que habia creado para la funcion: 

In [41]:
df_itemsf2 = pd.read_parquet(r'DataApi/items_f2.parquet')

* reduzco el df de items para trabajar eficientemente ya que el consumo de memoria es alto:

In [47]:
df_items_reducido = df_itemsf2[df_itemsf2['item_id'].isin(df_gamesf1['id'])]

In [53]:
df_items_reducido.reset_index(drop=True, inplace=True)

comparo el peso de los df items creado para ver cual es mas eficiente y trabajar con ese 

In [60]:
df_items_reducido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4294257 entries, 0 to 4294256
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           int32  
 1   playtime_forever  float32
 2   user_id           object 
dtypes: float32(1), int32(1), object(1)
memory usage: 65.5+ MB


In [52]:
df_itemsf2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           int32  
 1   playtime_forever  float32
 2   user_id           object 
dtypes: float32(1), int32(1), object(1)
memory usage: 78.6+ MB


dado que df_items_reducido tiene menor peso procedo a trabajar con el en mi funcion a continuacion:

In [ ]:
df_items_reducido = df_items_reducido.to_csv('games_f2.csv')

In [3]:
df_steam_games = pd.read_csv(r'DataApi/games_f2.csv')
df_user_items = pd.read_csv(r'DataApi/user_items.csv')

In [ ]:
df_game = df_steam_games.drop(['name', 'developer', 'price', 'tags','specs'],axis=1)

# Merge de los DataFrames
merged_data=pd.merge(df_user_items,df_game,left_on='item_id', right_on='item_id', how='right')

merged_data.drop(['item_id'], axis=1, inplace=True)
merged_data.dropna(inplace=True)

merged_data['hours_game'] = merged_data['hours_game'].astype(int)
merged_data['year'] = merged_data['year'].astype(int)

#Convertimos la columna genres a tipo lista
merged_data['genres'] = merged_data['genres'].apply(ast.literal_eval)

#Expandimos las listas en columnas
merged_data = merged_data.explode('genres').reset_index(drop=True)

#Encontrar el jugador con más horas jugadas por género y año
max_hours_df = merged_data.loc[merged_data.groupby(['genres', 'year'])['hours_game'].idxmax()]

#Filtrar las columnas necesarias
max_hours_df = max_hours_df[['genres', 'year', 'user_id', 'hours_game']]

#Segunda agrupación para obtener las horas jugadas por año para el usuario seleccionado
max_hours = max_hours_df.groupby(['genres', 'year', 'user_id'])['hours_game'].sum().reset_index()

max_hours.to_csv('UserForGenre.csv', index=False)

In [5]:
def UserForGenre(genero:str):
    consulta2 = pd.read_csv('UserForGenre.csv')
    
    genre_data = consulta2[consulta2['genres'] == genero]

    # Encontrar al usuario con más horas jugadas para ese género
    top_user = genre_data.loc[genre_data['hours_game'].idxmax()]['user_id']

    # Crear una lista de acumulación de horas jugadas por año
    hours_by_year = genre_data.groupby('year')['hours_game'].sum().reset_index()
  
    hours_by_year = hours_by_year.rename(columns={'year': 'Año', 'hours_game': 'Horas'})
    
    hours_list = hours_by_year.to_dict(orient='records')

    # Crear el diccionario de retorno
    result = {
        "Usuario con más horas jugadas para Género {}".format(genero): top_user,
        "Horas jugadas": hours_list
    }

    return result

@app.get("/UserForGenre/{genero}", tags=['UserForGenre'])
async def UserForGenre(genero: str):

    try:
        # Validación adicional para asegurarse de que el género no sea nulo o esté vacío
        if not genero or not genero.strip():
            raise HTTPException(status_code=422, detail="El parámetro 'genero' no puede ser nulo o estar vacío.")

        result = UserForGenre(genero)
        
        # Validación para verificar si el género existe en los datos
        if not result:
            raise HTTPException(status_code=404, detail=f"No se encontró información para el género '{genero}'.")
            
        return result
    
    except FileNotFoundError as e:
        raise HTTPException(status_code=500, detail=f"Error al cargar el archivo UserForGenre.csv: {str(e)}")
    except Exception as e:
        traceback.print_exc()
        raise HTTPException(status_code=500, detail=f"Error interno del servidor: {str(e)}")


In [8]:
UserForGenre('Action')

{'Usuario con más horas jugadas para género Action': 'Sp3ctre',
 'Horas jugadas': [{'Año': 1993, 'Horas': 0},
  {'Año': 1995, 'Horas': 217},
  {'Año': 1996, 'Horas': 0},
  {'Año': 1998, 'Horas': 0},
  {'Año': 1999, 'Horas': 44},
  {'Año': 2000, 'Horas': 70644},
  {'Año': 2001, 'Horas': 13},
  {'Año': 2002, 'Horas': 238},
  {'Año': 2003, 'Horas': 7673},
  {'Año': 2004, 'Horas': 127411},
  {'Año': 2005, 'Horas': 21339},
  {'Año': 2006, 'Horas': 896},
  {'Año': 2007, 'Horas': 112784},
  {'Año': 2008, 'Horas': 224},
  {'Año': 2009, 'Horas': 108326},
  {'Año': 2010, 'Horas': 78083},
  {'Año': 2011, 'Horas': 154896},
  {'Año': 2012, 'Horas': 378296},
  {'Año': 2013, 'Horas': 120461},
  {'Año': 2014, 'Horas': 130691},
  {'Año': 2015, 'Horas': 307511},
  {'Año': 2016, 'Horas': 29576},
  {'Año': 2017, 'Horas': 43327}]}

3) def **UsersRecommend**( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

In [4]:
df_user_reviews = pd.read_csv(r'DataApi/user_reviews.csv')

In [56]:
# Merge de los DataFrames
df_merged = pd.merge(df_user_reviews, df_steam_games, on='item_id', how='left')

# Filtrar las filas que cumplen con las condiciones
result = df_merged.loc[(df_merged['recommend'] == True) & (df_merged['sentiment_analysis'].isin([1, 2])),
                           ['year_x', 'name']]

# Renombrar la columna 'year_x' a 'year'
result = result.rename(columns={'year_x': 'year'})

# Agrupar por 'year' y 'name' y contar las ocurrencias
grouped_result = result.groupby(['year', 'name']).size().reset_index(name='count')

# Ordenar por 'year' y 'count' en orden descendente
grouped_result = grouped_result.sort_values(by=['year', 'count'], ascending=[False, False])

# Obtener el top 3 por año
top3_by_year = grouped_result.groupby('year').head(3)

# Guardar el resultado en un archivo CSV
top3_by_year.to_csv('UsersRecommend.csv', index=False)

In [ ]:
from fastapi.responses import JSONResponse


def UsersRecommend(year: int):
    df = pd.read_csv('UsersRecommend.csv')
    
    # Filtrar el DataFrame por el año especificado
    result_df = df[df['year'] == year]

    response_data = [{"Puesto 1": result_df.iloc[0]['name']},
                     {"Puesto 2": result_df.iloc[1]['name']},
                     {"Puesto 3": result_df.iloc[2]['name']}]

    return response_data
@app.get("/UsersRecommend/{year}", tags=['UsersRecommend'])
async def endpoint3(year: str):
    try:
        year = int(year)
    
        if not (2000 <= year <= 2100):
            error_message = f"El año debe estar en el rango entre 2000 y 2100 {str(e)}"
            return JSONResponse(status_code=500, content={"error": error_message})
        
        result = UsersRecommend(year)
    
        if result:
            return result
        else:
            #raise HTTPException(status_code=404, detail=f"No se encontraron recomendaciones para el año {year}.")
            error_message = "No se encontraron recomendaciones para el año {year} {str(e)}"
            return JSONResponse(status_code=500, content={"error": error_message})

    except FileNotFoundError as e:
        error_message = f"Error al cargar el archivo UsersRecommend.csv: {str(e)}"
        return JSONResponse(status_code=500, content={"error": error_message})

    except Exception as e:
        error_message = f"Error interno del servidor: {str(e)}"
        return JSONResponse(status_code=500, content={"error": error_message})


In [57]:
UsersRecommend(2010)

[{'Puesto 1': "Sid Meier's Civilization® V"},
 {'Puesto 2': 'Mount & Blade: Warband'},
 {'Puesto 3': 'Fallout: New Vegas'}]

4) def **UsersNotRecommend**( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

In [ ]:
df_games_f3 = pd.read_csv('df_games.csv')
df_reviews_f3 = pd.read_csv('df_reviews.csv')

In [10]:
def UsersNotRecommend(año: int):
    # Unir los DataFrames de juegos y reseñas en 'id'
    df_merged = pd.merge(df_games_f3, df_reviews_f3, left_on='id', right_on='item_id', how='inner')

    # Filtrar por reseñas negativas y por el año especificado
    df_filtered = df_merged[(df_merged['recommend'] == False) & (df_merged['sentiment_analysis'] == 0) & 
                            (pd.to_datetime(df_merged['release_date'], errors= 'coerce').dt.year == año)]

    if not df_filtered.empty:
        # Agrupar por juego y contar las no recomendaciones
        bottom_games = df_filtered.groupby('app_name')['recommend'].count().reset_index()

        # Ordenar de forma ascendente y tomar los primeros 3
        bottom_games = bottom_games.sort_values(by='recommend', ascending=True).head(3)

        # Crear la estructura de retorno
        retorno = [{"Puesto {}".format(i+1): juego} for i, juego in enumerate(bottom_games['app_name'])]

        return retorno
    else:
        return {"mensaje": "No hay juegos menos recomendados para el año especificado"}

In [11]:
UsersNotRecommend(2011)

[{'Puesto 1': 'Mafia II'},
 {'Puesto 2': 'L.A. Noire'},
 {'Puesto 3': 'Lord of the Rings: War in the North'}]

5) def **sentiment_analysis**( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

In [16]:
def sentiment_analysis(año: int):
    # uno DataFrame games y reviews en 'id'
    df_merged = pd.merge(df_gamesf1, df_reviews_f3, left_on='id', right_on='item_id', how='inner')

    # Filtrar por el año especificado
    df_filtered = df_merged[pd.to_datetime(df_merged['release_date'], errors='coerce').dt.year == año]

    if not df_filtered.empty:
        # Contar registros según el análisis de sentimiento
        sentiment_counts = df_filtered['sentiment_analysis'].value_counts().to_dict()

        # Mapear los valores para tener las etiquetas correspondientes
        sentiment_counts = {
            "Negative": sentiment_counts.get(0, 0),
            "Neutral": sentiment_counts.get(1, 0),
            "Positive": sentiment_counts.get(2, 0)
        }

        return sentiment_counts
    else:
        return {"mensaje": "No hay registros para el año especificado"}

In [17]:
sentiment_analysis(2011)

{'Negative': 462, 'Neutral': 716, 'Positive': 2232}